## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-18-16-52-14 +0000,bbc,UK set for talks over access to EU defence loans,https://www.bbc.com/news/articles/c07vdy4yz9do...
1,2025-09-18-16-51-25 +0000,nyt,Climate Activists Argue 3 Executive Orders Vio...,https://www.nytimes.com/2025/09/18/climate/lig...
2,2025-09-18-16-50-23 +0000,nyt,Live Updates: Trump and Starmer Acknowledge Di...,https://www.nytimes.com/live/2025/09/18/world/...
3,2025-09-18-16-48-58 +0000,bbc,First migrant deported to France under 'one in...,https://www.bbc.com/news/articles/ckg653r06jgo...
4,2025-09-18-16-48-24 +0000,startribune,Live: Feds announce charges related to housing...,https://www.startribune.com/live-feds-to-annou...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,59
58,kirk,25
57,charlie,22
48,kimmel,16
96,was,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
58,2025-09-18-15-10-04 +0000,nypost,Trump claims Jimmy Kimmel was taken off air fo...,https://nypost.com/2025/09/18/us-news/trump-cl...,175
9,2025-09-18-16-40-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...,140
131,2025-09-18-10-45-28 +0000,nypost,Vance describes ‘tough moment’ he broke news o...,https://nypost.com/2025/09/18/us-news/vance-de...,129
268,2025-09-17-23-03-00 +0000,wsj,Some military lawyers and other Defense Depart...,https://www.wsj.com/politics/national-security...,115
314,2025-09-17-20-29-18 +0000,nypost,Trump calls UK state dinner with King Charles ...,https://nypost.com/2025/09/17/world-news/presi...,114


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
58,175,2025-09-18-15-10-04 +0000,nypost,Trump claims Jimmy Kimmel was taken off air fo...,https://nypost.com/2025/09/18/us-news/trump-cl...
9,81,2025-09-18-16-40-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...
230,64,2025-09-18-01-41-00 +0000,wsj,When the Federal Reserve cut interest rates We...,https://www.wsj.com/economy/central-banking/fe...
315,62,2025-09-17-20-25-00 +0000,wsj,U.S. military operations against Latin America...,https://www.wsj.com/politics/national-security...
353,57,2025-09-17-18-21-00 +0000,wsj,Former CDC Director Susan Monarez rebutted Rob...,https://www.wsj.com/politics/policy/susan-mona...
314,55,2025-09-17-20-29-18 +0000,nypost,Trump calls UK state dinner with King Charles ...,https://nypost.com/2025/09/17/world-news/presi...
316,49,2025-09-17-20-22-19 +0000,cbc,Canada says Israel's 'horrific' Gaza City offe...,https://www.cbc.ca/news/politics/canada-gaza-c...
245,47,2025-09-18-00-14-00 +0000,wsj,Heightened fears of politically motivated atta...,https://www.wsj.com/politics/policy/rattled-la...
209,44,2025-09-18-03-38-00 +0000,wsj,President Trump is frustrated with Israeli Pri...,https://www.wsj.com/world/middle-east/trump-ne...
357,39,2025-09-17-18-00-01 +0000,latimes,"California, West Coast states roll out their o...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
